This notebook is just meant to test the codes before I compile them as .py files.

In [6]:
import numpy as np
import pandas as pd

# 1. Inspect Raw Data

In [13]:
# 6dFGS
df = pd.read_csv('data/raw/6dfgs/campbell_table2.ascii', delim_whitespace=True)

df.head()

,n,sdf,tmass,mjd,z_helio,s2n,r,cor_sigma,dex_error
0,1,g0000144-765225,2MASXJ00001440-7652248,53614.60,0.0533,13.2,14.0,129.0,0.073
1,2,g0000222-013746,2MASXJ00002213-0137463,52962.44,0.0383,24.1,18.4,199.1,0.023
2,3,g0000235-065610,2MASXJ00002348-0656103,52966.46,0.0376,15.2,16.6,156.2,0.045
3,4,g0000251-260240,2MASXJ00002509-2602401,52846.78,0.0508,13.2,13.0,218.5,0.040
4,5,g0000356-014547,2MASXJ00003564-0145472,52962.44,0.0244,14.0,16.3,133.4,0.066


In [14]:
df = pd.read_csv('data/raw/6dfgs/campbell_table8.ascii', delim_whitespace=True)

df.head()

,6dFGSid,2MASSid,ra,dec,cz,logRe_J,elogRe_J,logRe_H,elogRe_H,logRe_K,...,Nr,zgroup,dist5,sdens5,sampcodeJ,pvcodeJ,sampcodeH,pvcodeH,sampcodeK,pvcodeK
0,g0000144-765225,2MASXJ00001440-7652248,0.00399,-76.87364,15941,0.465,0.057,0.363,0.069,0.261,...,-1,-1,4.2758,0.1176,111111,1,111111,1,111111,1
1,g0000222-013746,2MASXJ00002213-0137463,0.00615,-1.62947,11123,0.024,0.048,-0.010,0.057,-0.087,...,-1,-1,6.4318,0.0385,111111,1,111111,1,111111,1
2,g0000235-065610,2MASXJ00002348-0656103,0.00652,-6.93619,10920,0.343,0.047,0.370,0.057,0.133,...,-1,-1,6.2142,0.0448,111111,1,111111,1,111111,1
3,g0000251-260240,2MASXJ00002509-2602401,0.00697,-26.04450,14926,0.371,0.046,0.338,0.055,0.310,...,-1,-1,3.2827,0.1434,111111,1,111111,1,111111,1
4,g0000356-014547,2MASXJ00003564-0145472,0.00990,-1.76317,6956,0.316,0.042,0.286,0.052,0.320,...,-1,-1,3.8994,0.1036,111111,1,111111,1,111111,1
